En aquesta pràctica treballareu amb un model de detecció d’objectes basat en PyTorch i la xarxa neuronal VGG16. L'objectiu es modificar el model per adaptar-lo a un problema de detecció d'objectes específic utilitzant un conjunt de dades simples.

![Exemple](08_Detecció/imgs/img.png)

Emprarem un *dataset* de detecció d'objectes que conté imatges d'estrelles. Aquest conjunt de dades és senzill i ideal per a practicar tècniques de detecció d'objectes. El podeu trobar a Kaggle al següent [enllaç](https://www.kaggle.com/datasets/kishanj/simple-object-detection). Per carregar aquest tipus de dataset haurem d'implementar una classe personalitzada que hereti de `torch.utils.data.Dataset`.

In [5]:
from torch.utils.data import Dataset

class EstrellesDataset(Dataset):
    def __init__(self, transforms):
        super().__init__()
        self.transform = transforms

        with open(annotation_file, "r") as f:
            self.annotations = json.load(f)

    def __len__(self):
        return len(self.annotations)


    def __getitem__(self, idx):
        # TODO
        sample = self.annotations[idx]

        img_path = os.path.join(self.root_dir, sample["image"])
        img = Image.open(img_path).convert("RGB")

        bbox = torch.tensor(sample["bbox"], dtype=torch.float32)
        label = torch.tensor(sample["label"], dtype=torch.long)

        if self.transform:
            img = self.transform(img)

        return img, label, bbox

## Preparació del model

Començarem carregant el model VGG16 preentrenat i adaptant-lo per a la detecció d'objectes. Afegirem capes addicionals per predir les caixes delimitadores (bounding boxes) i les classes dels objectes.

In [7]:
from torch import nn
from torchvision import models, transforms

vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
backbone = vgg16.features

class VGG16ObjectDetector(nn.Module):
    def __init__(self, backbone, num_classes=2):
        super().__init__()
        self.backbone = backbone
        self.avgpool =  nn.AdaptiveAvgPool2d((7, 7))# TODO

        # Capes fully-connected de la VGG original
        self.flatten = nn.Flatten()
        self.fc =  nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
        )

        # Cap de classificació
        self.class_head = nn.Linear(4096, num_classes)

        # Cap de regressió de bounding box
        self.bbox_head = nn.Linear(4096, 4)

    def forward(self, x):

        x = self.backbone(x)           # [B, 512, H', W'] (H',W' depèn d'H,W)
        x = self.avgpool(x)            # [B, 512, 7, 7]
        x = self.flatten(x)            # [B, 512*7*7]
        x = self.fc(x)                 # [B, 4096]

        class_logits = self.class_head(x)  # [B, num_classes]
        bbox_preds = self.bbox_head(x)

        return class_logits, bbox_preds

VGG16ObjectDetector(backbone=backbone)

VGG16ObjectDetector(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, paddi